<a href="https://colab.research.google.com/github/Rohith1-p/Handwritten-digit-recognition-using-kNN/blob/gh-pages/Car_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import pandas as pd

# Data Collection

In [ ]:
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Car_Price_Prediction/train_Y_cp.csv
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Car_Price_Prediction/train_X_cp.csv
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Car_Price_Prediction/test_X_cp.csv

--2021-06-19 09:31:53--  https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Car_Price_Prediction/train_Y_cp.csv
Resolving nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com)... 52.219.64.23
Connecting to nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com)|52.219.64.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 881 [text/csv]
Saving to: ‘train_Y_cp.csv’

train_Y_cp.csv      100%[===================>]     881  --.-KB/s    in 0s      

2021-06-19 09:31:54 (53.1 MB/s) - ‘train_Y_cp.csv’ saved [881/881]

--2021-06-19 09:31:54--  https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Car_Price_Prediction/train_X_cp.csv
Resolving nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-

In [ ]:
train_X_df = pd.read_csv('train_X_cp.csv')
train_Y_df = pd.read_csv('train_Y_cp.csv', header=None).squeeze()

test_X_df = pd.read_csv('test_X_cp.csv')

In [ ]:
train_X_df

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg
0,0,90.09,toyota,gas,std,four,hatchback,fwd,front,95.7,164.637,63.756,52.272,2109,ohc,four,98,2bbl,3.19,3.03,9.0,70.0,4800.0,30,37
1,0,84.15,subaru,gas,turbo,four,wagon,4wd,front,96.9,171.864,64.746,54.351,2650,ohcf,four,108,mpfi,3.62,2.64,7.7,111.0,4800.0,23,23
2,1,126.72,nissan,gas,std,two,hatchback,fwd,front,94.5,163.944,63.162,52.767,2028,ohc,four,97,2bbl,3.15,3.29,9.4,69.0,5200.0,31,37
3,0,77.22,honda,gas,std,four,wagon,fwd,front,96.5,155.529,63.261,57.717,2024,ohc,four,92,1bbl,2.92,3.41,9.2,76.0,6000.0,30,34
4,-1,73.26,volvo,gas,std,four,wagon,rwd,front,104.3,186.912,66.528,56.925,3034,ohc,four,141,mpfi,3.78,3.15,9.5,114.0,5400.0,23,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,1,166.32,toyota,gas,std,two,hatchback,rwd,front,94.5,167.013,63.360,52.074,2300,dohc,four,98,mpfi,3.24,3.08,9.4,112.0,6600.0,26,29
156,0,NaN,volkswagen,gas,std,four,wagon,fwd,front,100.4,181.269,66.231,54.549,2563,ohc,four,109,mpfi,3.19,3.40,9.0,88.0,5500.0,25,31
157,3,NaN,plymouth,gas,turbo,two,hatchback,rwd,front,95.9,171.468,65.637,49.698,2818,ohc,four,156,spdi,3.59,3.86,7.0,145.0,5000.0,19,24
158,3,195.03,toyota,gas,std,two,hatchback,rwd,front,102.9,181.665,67.023,51.480,3016,dohc,six,171,mpfi,3.27,3.35,9.3,161.0,5200.0,19,24


In [ ]:
train_X_df['drive-wheels'].replace(to_replace = '4wd',value = 'fwd',inplace = True)

In [ ]:
train_X_df['drive-wheels'].unique()

array(['fwd', 'rwd'], dtype=object)

#**UNDERSTANDIG THE DATA**

In [ ]:
print('train_X_df shape',train_X_df.shape)
print('train_Y_df shape',train_Y_df.shape)
print('test_X_df',test_X_df.shape)

train_X_df shape (160, 25)
train_Y_df shape (160,)
test_X_df (41, 25)


In [ ]:

train_X_df.isna().sum()

symboling             0
normalized-losses    30
make                  0
fuel-type             0
aspiration            0
num-of-doors          2
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type           0
num-of-cylinders      0
engine-size           0
fuel-system           0
bore                  4
stroke                4
compression-ratio     0
horsepower            2
peak-rpm              2
city-mpg              0
highway-mpg           0
dtype: int64

In [ ]:
categorical_cols = train_X_df.columns[train_X_df.dtypes == 'object']
categorical_cols

Index(['make', 'fuel-type', 'aspiration', 'num-of-doors', 'body-style',
       'drive-wheels', 'engine-location', 'engine-type', 'num-of-cylinders',
       'fuel-system'],
      dtype='object')

In [ ]:
numerical_cols = set(train_X_df.columns) - set(categorical_cols)
numerical_cols = list(numerical_cols)
numerical_cols

['length',
 'wheel-base',
 'highway-mpg',
 'city-mpg',
 'height',
 'stroke',
 'normalized-losses',
 'bore',
 'compression-ratio',
 'symboling',
 'peak-rpm',
 'width',
 'curb-weight',
 'engine-size',
 'horsepower']

In [ ]:
train_X_df.dtypes

symboling              int64
normalized-losses    float64
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                 float64
stroke               float64
compression-ratio    float64
horsepower           float64
peak-rpm             float64
city-mpg               int64
highway-mpg            int64
dtype: object

In [ ]:
train_X_df['symboling'].unique()

array([ 0,  1, -1,  2,  3, -2])

#**Discrete  cols**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
discrete_cols = ['symboling']
discrete_pipe = Pipeline(steps = [ ('discrete_imputer',SimpleImputer(strategy = 'median'))])

#**Continuous cols**

In [ ]:
continuous_cols = list(set(numerical_cols) - set(discrete_cols))
continuous_cols
continuous_pipe = Pipeline(steps = [ ('continuous_imputer',SimpleImputer(strategy = 'mean'))])

#**Categrical cols**

In [ ]:
categorical_cols

Index(['make', 'fuel-type', 'aspiration', 'num-of-doors', 'body-style',
       'drive-wheels', 'engine-location', 'engine-type', 'num-of-cylinders',
       'fuel-system'],
      dtype='object')

#**Ordinal cols**

In [ ]:
ordinal_cols = ['num-of-doors','num-of-cylinders']

print(train_X_df['num-of-doors'].unique())
print(train_X_df['num-of-cylinders'].unique())

['four' 'two' nan]
['four' 'six' 'five' 'eight' 'three' 'two' 'twelve']


In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_cols

ordinal_categories = [['num-of-doors','two','four' ],
                       ['num-of-cylinders','two','three','four','five','six','eight','twelve']]
ordinal_pipe = Pipeline(steps= [('ordinal_imputer',SimpleImputer(strategy= 'most_frequent')),
                                ('ordinal_encoder', OrdinalEncoder(categories = ordinal_categories))])

#Nominal data

In [ ]:
nominal_cols = list(set(categorical_cols)- set(ordinal_cols))
from sklearn.preprocessing import OneHotEncoder

nominal_pipe = Pipeline(steps = [('encoder',OneHotEncoder(handle_unknown = 'ignore'))])

In [ ]:
train_X_df[nominal_cols].isna().sum()

body-style         0
aspiration         0
fuel-system        0
make               0
fuel-type          0
engine-location    0
drive-wheels       0
engine-type        0
dtype: int64

#**Column Transformer**

In [ ]:
from sklearn.compose import ColumnTransformer

column_transformer = ColumnTransformer(transformers= [('discrete', discrete_pipe,discrete_cols),
                                                      ('continuous',continuous_pipe,continuous_cols),
                                                      ('ordinal',ordinal_pipe,ordinal_cols),
                                                      ('nominal',nominal_pipe,nominal_cols)],
                                                      remainder = 'passthrough')

# Implementation

In [ ]:
# ADD YOUR CODE HERE
from sklearn.linear_model import Ridge, Lasso,LinearRegression
from sklearn.preprocessing import RobustScaler,MinMaxScaler, StandardScaler
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
pipeline = Pipeline(steps = [('preprocessor',column_transformer),
                             ('scaler',StandardScaler()),
                             ('selector', SelectKBest(score_func=f_regression,k = 35)),
                             ('estimator',LinearRegression())])

In [ ]:
from sklearn.metrics import make_scorer , accuracy_score 
param_distributions = [{
                        'estimator':[LinearRegression()],
                        
                        },
                       {
                        'estimator': [Ridge(alpha =20.0,solver= 'saga',max_iter= 2500)]
                        },
                       {
                        'estimator': [Lasso(alpha = 2.0,max_iter= 2000)]}]

In [ ]:
random_search_cv = RandomizedSearchCV(pipeline,param_distributions,refit=True,cv = 5)
random_search_cv.fit(train_X_df,train_Y_df)
print(random_search_cv.best_params_)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


{'estimator': Ridge(alpha=20.0, copy_X=True, fit_intercept=True, max_iter=2500,
      normalize=False, random_state=None, solver='saga', tol=0.001)}


In [ ]:
best_model = random_search_cv.best_estimator_
best_model.fit(train_X_df,train_Y_df)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('discrete',
                                                  Pipeline(memory=None,
                                                           steps=[('discrete_imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                        

In [ ]:
predicted_test_Y = best_model.predict(test_X_df)

**Cross Validation with pipeline**

# Submission 
Writing the predicted values to CSV file

In [ ]:
pd.DataFrame(predicted_test_Y).to_csv('predicted_test_Y_cp.csv', header=None, index=False)